## Clean your deployed model endpoint content
-----

#### In this notebook, we will parse through the existing endpoint.json to delete all of the existing endpoints once you are done with running your respective benchmarking tests.

***If you are with running all of the tests, and want to delete the existing endpoints, run this notebook.***

In [7]:
# if interactive mode is set to no -> pickup fmbench from Python installation path
# if interactive mode is set to yes -> pickup fmbench from the current path (one level above this notebook)
# if interactive mode is not defined -> pickup fmbench from the current path (one level above this notebook)
# the premise is that if run non-interactively then it can only be run through main.py which will set interactive mode to no
import os
import sys
import subprocess
from pathlib import Path
import importlib.resources as pkg_resources
if os.environ.get("INTERACTIVE_MODE_SET", "yes") == "yes":
    sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
## Import all necessary libraries
import json
import boto3
import logging
from fmbench.utils import *
from fmbench.globals import *

In [ ]:
# Set your logger to display all of the endpoints being cleaned
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

# Load the config file to extract our endpoint.json file and its respective file path
config = load_main_config(CONFIG_FILE)

In [ ]:
# Refer to the file path for the endpoint
endpoint_info_list = json.loads(get_s3_object(config['aws']['bucket'], ENDPOINT_LIST_PATH))
logger.info(f"found information for {len(endpoint_info_list)} endpoints, bucket={config['aws']['bucket']}, key={ENDPOINT_LIST_PATH}")
logger.info(json.dumps(endpoint_info_list, indent=2))

In [ ]:
# initialize a sagemaker client
sm_client = boto3.client("sagemaker")
config['experiments']
ep_names_from_config_not_deployed = [e['ep_name'] for e in config['experiments'] if e['deploy'] is False]

# check if this is deployedon EC2, if yes then skip SageMake portion
all_ep_on_ec2 = all([True for e in config['experiments'] if e.get('ec2')])
if all_ep_on_ec2 is False:
    # there are some instances not on EC2, so maybe try SageMaker
    # Iterate over the endpoint_info_list and mark the items for deletion
    for item in endpoint_info_list:   
        
        # Extract the endpoint name from the deployed model configuration
        ep_name = item['endpoint']["EndpointName"]
        # check if this endpoint was not actually deployed by fmbench and if not 
        # then we should not be deleting it
        if ep_name in ep_names_from_config_not_deployed:
            logger.info(f"{ep_name} is in ep_names_from_config_not_deployed, not deleting it")
            continue
        try:
            ## Describe the model endpoint 
            logger.info(f"Going to describe the endpoint -> {ep_name}")
            resp = sm_client.describe_endpoint(EndpointName=ep_name)

            ## If the given model endpoint is in service, delete it 
            if resp['EndpointStatus'] == 'InService':
                logger.info(f"going to delete {ep_name}")
                ## deleting the model endpoint
                sm_client.delete_endpoint(EndpointName=ep_name)
                logger.info(f"deleted {ep_name}")
        except Exception as e:
            logger.error(f"error deleting endpoint={ep_name}, exception={e}")
else:
    logger.info(f"all_ep_on_ec2={all_ep_on_ec2}, skipping SageMaker portion")

In [ ]:
# Check and delete EKS resources based on manifest files and deployment status
for experiment in config['experiments']:
    if experiment['deploy'] and 'eks' in experiment:
        manifest_folder_path = Path(pkg_resources.files('fmbench'), experiment['eks']['manifest_dir'])
        eks_manifest_ray_path = os.path.join(manifest_folder_path, experiment['eks']['manifest_file'])
        logger.info(f"Deploy is set to True for {experiment['name']}, checking for manifest file {eks_manifest_ray_path}")
        try:
            logger.info(f"Cleaning up {experiment['name']} using kubectl")
            delete_command_args = ["kubectl", "delete", "-f", eks_manifest_ray_path]
            delete_result = subprocess.run(delete_command_args, capture_output=True, text=True)
            logger.info(f"Deleted the EKS endpoint: {delete_result.stdout}")
            logger.info(f"Cleaned up {experiment['name']} using kubectl")
        except Exception as e:
            logger.error(f"Error cleaning up manifest file for {experiment['name']}, exception={e}")
    else:
        continue